# Lucas-Kanade Optical Flow in OpenCV 

In [52]:
import numpy as np
import cv2 as cv
import argparse

image = 'slow_traffic_small.mp4'

cap = cv.VideoCapture(image)

# params pour la détection des coins ShiTomasi
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Paramètres pour le flux optique lucas kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Créez des couleurs aléatoires
color = np.random.randint(0,255,(100,3))

# Prenez la première image et trouvez-y des coins
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Créer une image de masque à des fins de dessin
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    if ret == False:
        break
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
        
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
        
    img = cv.add(frame,mask)
    cv.imshow('frame',img)
    
    
    
    # Hit 'q' on the keyboard to quit!
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
    
# Release handle to the webcam
cap.release()
cv.destroyAllWindows()

# Dense Optical Flow in OpenCV 

In [41]:
import numpy as np
import cv2 as cv

#cap = cv.VideoCapture(cv.samples.findFile("dataset/videoplayback.mp4"))
cap = cv.VideoCapture(cv.samples.findFile("dataset/Venice-1-raw.mp4"))

ret, frame1 = cap.read()

prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)

hsv[...,1] = 255

while(1):
    ret, frame2 = cap.read()
    if ret == False:
        break
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    
    height, width, _ = np.array(frame2).shape
    pourcentage_h = 0.7
    pourcentage_w = 0.4
    
    final = vis = np.concatenate((cv.resize(frame2, (int(height*pourcentage_h), int(width*pourcentage_w))),
                                  cv.resize(bgr, (int(height*pourcentage_h), int(width*pourcentage_w)))), axis=1)
    
    cv.imshow('frame2',final)
    
    # Hit 'q' on the keyboard to quit!
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    if cv.waitKey(1) & 0xFF == ord('a'):
        cv.imwrite('opticalfc_2.png',final)
        
    prvs = next
    
# Release handle to the webcam
cap.release()
cv.destroyAllWindows()

# 1. Estimation du flot dans une séquence vidéo

### 1.1. Flow optique en utilisant la norme

Plus l'intensité lumineuse est élevée, plus la norme du déplacement est élevée (Plus l'objet bouge rapidement)

In [257]:
import numpy as np
import cv2 as cv


cap = cv.VideoCapture(cv.samples.findFile("PETS09-S2L1-raw.mp4"))

ret, frame1 = cap.read()

prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)

hsv[...,0] = 0
hsv[...,1] = 0

#i = 0
#fourcc = cv.VideoWriter_fourcc(*'XVID')
#out = cv.VideoWriter('norme.avi', fourcc, 7.0, (978, 499))

while(1):
    ret, frame2 = cap.read()
    if ret == False:
        break
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 10, 5, 7, 1.5, 0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    #hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    
    height, width, _ = np.array(frame2).shape
    pourcentage_h = 0.85
    pourcentage_w = 0.65
    
    final = vis = np.concatenate((cv.resize(frame2, (int(height*pourcentage_h), int(width*pourcentage_w))),
                                  cv.resize(bgr, (int(height*pourcentage_h), int(width*pourcentage_w)))), axis=1)
    
    cv.imshow('frame2',final)
    #if i > 570 and i <= 620:
    #    out.write(final)
    #i += 1
    #if i > 620:
    #    break
    
    # Hit 'q' on the keyboard to quit!
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    if cv.waitKey(1) & 0xFF == ord('a'):
        cv.imwrite('opticalfc.png',final)
        #cv.imwrite('opticalhsv.png',bgr)
        
    prvs = next
    
# Release handle to the webcam
cap.release()
#out.release()
cv.destroyAllWindows()

### 1.2. Flow optique en utilisant la norme + la direction

- Plus l'intensité lumineuse est élevée, plus la norme du déplacement est élevée (Plus l'objet bouge rapidement)
- La couleur correspond à l'orientation

In [258]:
import numpy as np
import cv2 as cv


cap = cv.VideoCapture(cv.samples.findFile("PETS09-S2L1-raw.mp4"))

ret, frame1 = cap.read()

prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)

hsv[...,0] = 255
hsv[...,1] = 255

#i = 0
#fourcc = cv.VideoWriter_fourcc(*'XVID')
#out = cv.VideoWriter('norme.avi', fourcc, 7.0, (978, 499))

while(1):
    ret, frame2 = cap.read()
    if ret == False:
        break
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 10, 5, 7, 1.5, 0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    
    height, width, _ = np.array(frame2).shape
    pourcentage_h = 0.85
    pourcentage_w = 0.65
    
    final = vis = np.concatenate((cv.resize(frame2, (int(height*pourcentage_h), int(width*pourcentage_w))),
                                  cv.resize(bgr, (int(height*pourcentage_h), int(width*pourcentage_w)))), axis=1)
    
    cv.imshow('frame2',final)
    #if i > 570 and i <= 620:
    #    out.write(final)
    #i += 1
    #if i > 620:
    #    break
    
    # Hit 'q' on the keyboard to quit!
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    if cv.waitKey(1) & 0xFF == ord('a'):
        cv.imwrite('opticalfc.png',final)
        #cv.imwrite('opticalhsv.png',bgr)
        
    prvs = next
    
# Release handle to the webcam
cap.release()
#out.release()
cv.destroyAllWindows()